### DataFrame API — практика

Вы прошли тему, посвящённую DataFrame API. Теперь попрактикуйтесь в Spark и проанализируйте уже знакомые [данные о жилье в Калифорнии](https://www.kaggle.com/bhavinmoriya/introduction-to-pyspark/data). С этим датасетом вы будете работать и в самостоятельном проекте.

Данные, которые вы будете анализировать, были собраны в рамках переписи населения в США. Каждая строка содержит агрегированную статистику о жилом массиве. Жилой массив — минимальная географическая единица с населением от 600 до 3000 человек в зависимости от штата. Одна строка в данных содержит статистику в среднем о 1425.5 обитателях жилого массива.

В колонках датасета содержатся следующие данные:

- `longitude` — широта;
- `latitude` — долгота;
- `housing_median_age` — медианный возраст жителей жилого массива;
- `total_rooms` — общее количество комнат в домах жилого массива;
- `total_bedrooms` — общее количество спален в домах жилого массива;
- `population` — количество человек, которые проживают в жилом массиве;
- `households` — количество домовладений в жилом массиве;
- `median_income` — медианный доход жителей жилого массива;
- `median_house_value` — медианная стоимость дома в жилом массиве;
- `ocean_proximity` — близость к океану.

В большинстве колонок хранятся количественные данные, кроме одной — `ocean_proximity`. Она хранит категориальные значения.

In [ ]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

**Задание 1.** Инициализируйте сессию и прочитайте данные из файла `/datasets/housing.csv` с помощью pySpark.

*Подсказка:* используйте метод для чтения файлов, передав ему аргумент `inferSchema=True`, чтобы Spark смог автоматически распознать названия и типы колонок в таблице.

In [ ]:
df_housing = spark.read.load('/datasets/housing.csv', format="csv", sep=",", inferSchema=True, header="true")
df_housing.printSchema()
df_housing.show(2)

Теперь можно переходить к анализу.

**Задание 2.** Выведите названия колонок и их тип данных в виде таблицы, используя атрибут `dtypes`. Представьте результат в виде таблицы pandas. Также выведите первые 10 строк датасета методом DataFrame API.

*Подсказка:* используйте атрибут таблицы `dtypes`, чтобы вывести названия колонок и их типы. Метод `show()` покажет первые строки датасета. Ему нужно передать количество строк, которое вы хотите вывести.

In [ ]:
# выведите названия колонок
print(pd.DataFrame(df_housing.dtypes, columns=['column', 'type']).head(10))

# выведите первые 10 строк
df_housing.show(10)

Красота! Теперь можно переходить к описательным статистикам.

**Задание 3.** Выведите с помощью метода `describe()` базовые описательные статистики данных в виде таблицы в pandas.

*Подсказка:* используйте метод `describe()`, аналогичный методу в pandas. Метод `describe()` — это трансформация, поэтому, чтобы посчитать статистики, примените действие `toPandas()`.

In [ ]:
# выведите базовые статистики
df_housing.toPandas().describe()

Ого, в датасете примерно 21 тысяча строк. Максимальная медианная стоимость дома равна полумиллиону долларов. Только данные получены давно — сейчас дома в Калифорнии стоят в несколько раз дороже.

**Задание 4.** Проверьте, есть ли домовладения, в которых медианная стоимость дома больше полумиллиона долларов.

*Подсказка:* чтобы посчитать домовладения, используйте методы `select()` и `count()`. Чтобы найти количество домовладений дороже 500000 долларов, вместе с `count()` используйте метод `filter()`, отфильтровав данные по колонке c медианной стоимостью.

In [ ]:
print('Количество домовладений: ', df_housing.select("households").count())
print('Количество домовладений где медианная стоимость больше $500000:', df_housing.filter(df_housing.median_house_value > 500000).count())

**Задание 5.** Проверьте, есть ли пропущенные значения в датасете. Найдите их количество в каждой колонке.

*Подсказка:* добавьте проверку колонки в переменную `check_col`. Приведите значения к типу `float` методом `cast()` и проверьте на наличие значений `[None, np.nan, NULL]`, используя метод `isin()`. Затем передайте методу `filter()` переменную с проверкой и выведите количество пропущенных значений.

In [ ]:
#Проверить с чем работаю
print(type(df_housing))

In [ ]:
# выведите пропущенные значения в каждой колонке
columns = df_housing.columns

for column in columns:
    check_col = F.col(column).cast("float").isNull()
    print(column, df_housing.filter(check_col).count())

In [ ]:
#Моя Проверка тест
check_col_2 = F.col('ocean_proximity').isNull()
print(df_housing.filter(check_col_2).count())

In [ ]:
#Моя Проверка 2 тест
# Подсчет количества вхождений каждого уникального значения в колонке 'ocean_proximity'
value_counts = df_housing.groupBy("ocean_proximity").count()

# Вывод результата
value_counts.show()

**Задание 6.** Найдите жилые массивы, в которых живут самые обеспеченные люди. Сгруппируйте данные по столбцу с категориями удалённости от океана и посчитайте среднее значение медианного дохода в жилом массиве.

Результат выведите таблицей в pandas. Нарисуйте столбчатый график с помощью библиотеки Seaborn.

*Подсказка:* примените метод `groupBy()` к датасету `df_housing`, передав методу колонку `ocean_proximity`. Добавьте к цепочке трансформаций метод `agg()` и передайте ему функцию для расчёта среднего `F.avg()` из модуля Spark Functions. Передайте функции нужную колонку.
В конце примените действие `toPandas()` к вашей цепочке трансформаций и создайте новую таблицу `df_wealthy`.
Для отрисовки графика по оси X укажите удалённость от океана, а по оси Y — среднее значение медианного дохода в жилом массиве.

In [ ]:
import seaborn as sns

df_wealthy = df_housing.groupBy("ocean_proximity").agg(F.avg('median_income').alias('median_income')).toPandas()
sns.barplot(x='ocean_proximity', y='median_income', data=df_wealthy)


Ожидаемый результат: люди с высоким доходом предпочитают селиться в жилых массивах, которые находятся не более чем в часе езды до океана или около залива.

**Задание 7.** Исследуйте зависимость возраста жителей от удалённости жилого массива от океана. Сгруппируйте данные по двум столбцам: медианный возраст жителей и удалённость от океана. Посчитайте, сколько раз встречается то или иное медианное значение возраста в зависимости от удалённости от океана.

Результат выведите таблицей в pandas. Нарисуйте линейный график с помощью библиотеки Seaborn.

*Подсказка:* примените метод `groupBy()` к датасету  `df_housing`, передав методу колонки с медианным возрастом жителей и удалённостью от океана. Добавьте к цепочке трансформаций метод `count()`. Передайте методу нужную колонку.
В конце примените действие `toPandas()` к вашей цепочке трансформаций и создайте новую таблицу `df_ages`.
Для отрисовки графика по оси X укажите медианный возраст в домовладении, по оси Y — количество домовладений, а в параметре цвета укажите удалённость от океана.

In [ ]:
df_ages = df_housing.groupBy("housing_median_age", "ocean_proximity").agg(F.count('population').alias('count')).toPandas()

sns.lineplot(data=df_ages,
    x="housing_median_age",
    y="count",
    hue="ocean_proximity"
)

По графику можно предположить, что люди до 20 лет чаще живут в материковой части Калифорнии (`INLAND`) и в радиусе часа езды до океана. Старшие поколения предпочитают селиться в радиусе часа езды до океана и около залива.

**Задание 8.** Создайте несколько новых столбцов с признаками:

- Отношение количества комнат `total_rooms` к количеству домовладений `households`. Назовите колонку `rooms_per_household`.
- Отношение количества жителей `population` к количеству домовладений `households`. Назовите колонку `population_in_household`.
- Отношение количества спален `total_bedrooms` к общему количеству комнат `total_rooms`. Назовите колонку `bedroom_index`.

*Подсказка:* для создания колонок используйте метод `withColumn()`. Первым аргументом укажите название новой колонки, а вторым — отношение двух колонок. Для выбора колонки используйте функцию `F.col()` из модуля Spark Functions.

In [ ]:
df_housing = df_housing.withColumn('rooms_per_household', F.col('total_rooms')/F.col('households'))
df_housing = df_housing.withColumn('population_in_household', F.col('population')/F.col('households'))
df_housing = df_housing.withColumn('bedroom_index', F.col('total_bedrooms')/F.col('total_rooms'))
df_housing.printSchema()

Отлично! Вы создали новые колонки, которые могут быть подходящими признаками для модели машинного обучения.

**Задание 9.** Выберите колонки со всеми признаками и запишите их в файл. Он пригодится вам в самостоятельном проекте.

Примените метод `select()` к датасету  `df_housing`, передав методу список нужных колонок. Далее примените методы `write()` и `format()` с аргументом `'csv'`. В конце добавьте метод `save()` и передайте ему путь для сохранения файла `path` и аргумент `header=True`, чтобы сохранить названия колонок.


In [ ]:
exclude = ['longitude', 'latitude']
# list comprehension — способ создания списков
selected_columns = [col for col in df_housing.columns if col not in exclude]
path = './california_housing_w_features'

df_housing.select(selected_columns) \
           .write \
           .format('csv') \
           .save('./california_housing_w_features', header=True)